In [1]:
# Import dependencies
import subprocess
import json
from dotenv import load_dotenv
import os
from web3 import Web3, HTTPProvider
from web3.middleware import geth_poa_middleware
from eth_account import Account
from bit import PrivateKeyTestnet, PrivateKey, wif_to_key
from bit.network import NetworkAPI

# Load and set environment variables
load_dotenv("keys.env")
mnem_user = os.getenv("MNEMONIC")

# Import constants.py and necessary functions from bit and web3
from constants import *

Python-dotenv could not parse statement starting at line 3


In [2]:
# establish ETH connection
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [3]:
# Create a function called `derive_wallets`
def derive_wallets(crypto):
    command = './derive -g --mnemonic=mnem_user --cols=address,index,path,privkey,pubkey,pubkeyhash,xprv,xpub --format=json --coin={} --numderive=3'.format(crypto)
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

In [4]:
coins = {'btctest': derive_wallets(BTCTEST), 'eth': derive_wallets(ETH), 'btc': derive_wallets(BTC)}
coins

{'btctest': [{'address': 'mmFJr4Yw2BBbw9UhnpcAPfd5PJ15YcPA7U',
   'index': 0,
   'path': "m/44'/1'/0'/0/0",
   'privkey': 'cUgTQ9A65vMLMc7vhdKprdWkvmba5fYPpByv9twd6YsNEfJ793bQ',
   'pubkey': '0250a50e4fa706741e1d31377dd55f924e328f4857ccdf9ec3151c2b66fff2f24d',
   'pubkeyhash': '3edb415129c56aaa22b78e66004f563452750bb2',
   'xprv': 'tprv8k1mxJukzjUFUaqFSujB4ZgTCFn5xv9CAE5njBAWW9LcVrU5YoytyrqA6E1A8xB4DAovHQX26xfvs9JQ3t3oyBJqTVzyHoTqBZeJ1zjexhU',
   'xpub': 'tpubDGhp6ix1979vN3s3LZPmTyLZmHJ28FL6jXga1hCovR91LLirBCoVAMT2GKxHMNeyquX69nQG6GfQVydK7qeJ7GcCkrMXzk5KzUuFGjFCh9m'},
  {'address': 'n3knqtx6N9DZzNZ1u84vQCBRJafHx5Xv59',
   'index': 1,
   'path': "m/44'/1'/0'/0/1",
   'privkey': 'cPNy7qvGf9Z1kC3JaYxGDF7jMpnAMM86G7oZL7HtKEEoBzrubvrJ',
   'pubkey': '03e2ada5948ded5f3cee3f1f94b4cdd900e99433f518f1d2d6f7c0ea4294b5558e',
   'pubkeyhash': 'f3f0b3901ac80efd186f259fd392c85d548d244a',
   'xprv': 'tprv8k1mxJukzjUFWYKDctH5nzQjePR4mm15Y3fVmuUf9DY4YHkMar5CkiADxuAvrapr7qUC3TZ8NNrURjuBBRMaqSvqVu7c6kvoFH

In [5]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST or coin == BTC:
        return PrivateKeyTestnet(priv_key)
    else:
        print("Invalid coin type")

In [6]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, to, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas(
        {"from": account.address, "to": to, "value": amount}
        )
        return {
            "from": account.address,
            "to": to,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
#             "chainID": 2022
        }
    elif coin == BTCTEST or coin == BTC:
        return PrivateKeyTestnet.prepare_transaction(account.address, 
                                                     [(to, amount, BTC)])
    else:
        print("Invalid coin")

In [7]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, to, amount):
    raw_tx = create_tx(coin, account, to, amount)
    if coin == ETH:
        signed = account.sign_transaction(raw_tx)
        return w3.eth.sendRawTransaction(signed.rawTransaction)
    elif coin == BTCTEST or coin == BTC:        
        signed = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed)
    else:
        print("Invalid coin")
    print(result.hex())    
    return result.hex()    

In [175]:
# BTCTEST tx
ac = priv_key_to_account(BTCTEST, coins['btctest'][2]['privkey'])
ad = coins['btctest'][1]['address']
send_tx(BTCTEST, ac, ad, 0.0001)

In [139]:
# ETH tx
ac = priv_key_to_account(ETH, coins['eth'][0]['privkey'])
ad = coins['eth'][1]['address']
send_tx(ETH, ac, ad, 100)

HexBytes('0x0329fb114fb577ff8d836db0a1ebe38206d790b4303160296bdf9bc61d1d754e')

In [8]:
# function to get address balances
def balances(coin, address, privkey):
    if coin == ETH:
        return w3.eth.getBalance(address)
    elif coin == BTCTEST or coin == BTC: 
        key = PrivateKeyTestnet(privkey)
        return key.get_balance()

In [9]:
balances(BTCTEST, coins['btctest'][2]['address'], coins['btctest'][2]['privkey'])

'3990381'

In [10]:
# function to get ETH transaction status
def tx_status(txid):
    return w3.eth.getTransaction(txid)

In [11]:
tx_status(0x0329fb114fb577ff8d836db0a1ebe38206d790b4303160296bdf9bc61d1d754e)

AttributeDict({'hash': HexBytes('0x0329fb114fb577ff8d836db0a1ebe38206d790b4303160296bdf9bc61d1d754e'),
 'nonce': 0,
 'blockHash': HexBytes('0x394b896e85fd6f50650c34fcc6b207958b42f65d27e4b6c869af08925c3ee688'),
 'blockNumber': 2,
 'transactionIndex': 0,
 'from': '0x11F86C79f55a28774Ec78AaF7F6C5cD8204eFC64',
 'to': '0x60Ef93b2493B10bF60A459C61466b767ECa75Fc7',
 'value': 100,
 'gas': 21000,
 'gasPrice': 20000000000,
 'input': '0x',
 'v': 27,
 'r': HexBytes('0xbfaa1dee707d7302879778177ac40adc561cc3c350b5a633a64777775f9d4df9'),
 's': HexBytes('0x020e945ae6c39ea2a7a98d3bd8b061c4443681b1f7fc3b8b5506848d44806523')})